In [5]:
import panel as pn
pn.extension()
import pandas as pd
import numpy as np
import hvplot.pandas
import param, random

In [6]:
B = pn.widgets.FloatSlider(name='Beta Distribution Parameter', start=0.0, end=1, step=0.02, value=0.01)
N = pn.widgets.FloatSlider(name='Number of Members', start=5, end=100, step=1, value=5)
P = pn.widgets.FloatSlider(name='Principle Working Capital (DAI)', start=5000, end=1000000, step=5000, value=5000)
D = pn.widgets.FloatSlider(name='Weekly Membership Dividends (Capital %)', start=0, end=5, step=0.05, value=0)
F = pn.widgets.FloatSlider(name='Weekly Membership Fee (DAI)', start=0, end=10, step=1, value=0)
R = pn.widgets.FloatSlider(name='Weekly Revenue (DAI)', start=0, end=10000, step=500, value=0)
E = pn.widgets.FloatSlider(name='Weekly Expenses (DAI)', start=0, end=10000, step=500, value=0)
Y = pn.widgets.FloatSlider(name='Years', start=1, end=20, step=1, value=1)

def beta_dist(B, N):
    xs = range(1,N)
    y = [B*(1-B)**t for t in xs]
    # Convert to a dataframe and return
    distribution = pd.DataFrame([xs,y]).T
    distribution.columns = ['Ranking','Ownership']
    # Force dist to sum to one
    distribution['Ownership'] += (1 - distribution['Ownership'].sum())/len(distribution)
    distribution.iloc[-1] += (1 - distribution['Ownership'].sum())
    
    return distribution

def capital_projection(N, P, D, F, R, E, Y):
    projections = []
    # Weekly projections
    for i in range(1,52*Y):
        P = (P+R+F*N-E)
        P = P - P*D/100
        projections.append(P)
    return pd.DataFrame(projections, columns=['Working Capital'])

@pn.depends(B, N, P, D, F, R, E, Y)
def dashboard(B, N, P, D, F, R, E, Y):
    distribution = beta_dist(B, N)
    distribution['Dividend Percentage'] = distribution['Ownership'] * D/100
    capital = capital_projection(N, P, D, F, R, E, Y)
    distribution["Average Weekly Payout (DAI)"] = distribution['Dividend Percentage'] * capital['Working Capital'].mean()
    distribution["Weekly Payouts (DAI)"] = distribution['Dividend Percentage'] * capital['Working Capital']
    return pn.Column(
                distribution.hvplot.line(x='Ranking',y='Ownership', title="Ownership Distribution"),
                capital.hvplot.line(xlabel='Weeks', title="Working Capital"),
                distribution.set_index("Ranking"),
            )


In [7]:
pn.Row(pn.Column(N,B,P,D,F,R,E, Y), dashboard)

Row
    [0] Column
        [0] FloatSlider(end=100, name='Number of Members', start=5, step=1, value=5, value_throttled=5)
        [1] FloatSlider(name='Beta Distribution P..., step=0.02, value=0.01, value_throttled=0.01)
        [2] FloatSlider(end=1000000, name='Principle Working C..., start=5000, step=5000, value=5000, value_throttled=5000)
        [3] FloatSlider(end=5, name='Weekly Membership D..., step=0.05, value_throttled=0)
        [4] FloatSlider(end=10, name='Weekly Membership F..., step=1, value_throttled=0)
        [5] FloatSlider(end=10000, name='Weekly Revenue (DAI)', step=500, value_throttled=0)
        [6] FloatSlider(end=10000, name='Weekly Expenses (DAI)', step=500, value_throttled=0)
        [7] FloatSlider(end=20, name='Years', start=1, step=1, value=1, value_throttled=1)
    [1] ParamFunction(function)

In [8]:
distribution = beta_dist(B.value, N.value)
distribution['Dividend Percentage'] = distribution['Ownership'] * D.value/100
capital = capital_projection(N.value, P.value, D.value, F.value, R.value, E.value, Y.value)
distribution["Average Weekly Payout (DAI)"] = distribution['Dividend Percentage'] * capital['Working Capital'].mean()
distribution["Weekly Payouts (DAI)"] = distribution['Dividend Percentage'] * capital['Working Capital']
distribution

,Ranking,Ownership,Dividend Percentage,Average Weekly Payout (DAI),Weekly Payouts (DAI)
0,1.0,0.250148,0.0,0.0,0.0
1,2.0,0.250049,0.0,0.0,0.0
2,3.0,0.249951,0.0,0.0,0.0
3,4.0,0.249853,0.0,0.0,0.0
